In [21]:
import os
import json
import pandas as pd
import traceback


In [22]:
from langchain.chat_models import ChatOpenAI


In [23]:
from dotenv import load_dotenv
load_dotenv()

True

In [24]:
KEY=os.getenv("OPENAI_API_KEY")

In [25]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)

In [26]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [27]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [28]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [29]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [30]:

quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\santh\AppData\Local\Temp\ipykernel_921716\3797420763.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [31]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [32]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [33]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [34]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [41]:
file_path=r"C:\Users\santh\mcqgen\data.txt"

In [42]:

with open(file_path, 'r') as file:
    TEXT = file.read()

In [44]:
print(TEXT)

Requirement for climate action which is around $2.3–2.5 trillion in emerging markets and developing countries (EMDCs) other than China. (Raising Ambition and Accelerating Delivery of Climate Finance' Report)
Due to inadequate funding, developing countries including India have rejected the climate finance deal agreed at COP29.
Deadlock on Mitigation Work Programme (MWP): Due to a divergence in views on the role of fossil fuels in the future energy mix.
The Global Stocktake, a comprehensive assessment of global progress towards climate goals, was a point of contention.
Limited progress on climate targets:  Progress in ambition and action since the initial NDCs plateaued and countries are still off track to deliver on the globally insufficient mitigation pledges for 2030. 
Global greenhouse gas emissions set a new record in 2023, with a 1.3% increase from 2022 levels (UNEP's Emissions Gap Report 2024).
Other Issues:
Slow pace of operationalizing the Loss and Damage Fund (LDF) and concerns

In [48]:

# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [51]:
NUMBER=5 
SUBJECT="environment"
TONE="simple"

In [53]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Requirement for climate action which is around $2.3–2.5 trillion in emerging markets and developing countries (EMDCs) other than China. (Raising Ambition and Accelerating Delivery of Climate Finance' Report)
Due to inadequate funding, developing countries including India have rejected the climate finance deal agreed at COP29.
Deadlock on Mitigation Work Programme (MWP): Due to a divergence in views on the role of fossil fuels in the future energy mix.
The Global Stocktake, a comprehensive assessment of global progress towards climate goals, was a point of contention.
Limited progress on climate targets:  Progress in ambition and action since the initial NDCs plateaued and countries are still off track to deliver on the globally insufficient mitigation pledges for 2030. 
Global greenhouse gas emissions set a new record in 2023, with a 1.3% increase from 2022 levels (UNEP's Emissio

In [54]:

print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1987
Prompt Tokens:1066
Completion Tokens:921
Total Cost:0.003441


In [55]:
response

{'text': "Requirement for climate action which is around $2.3–2.5 trillion in emerging markets and developing countries (EMDCs) other than China. (Raising Ambition and Accelerating Delivery of Climate Finance' Report)\nDue to inadequate funding, developing countries including India have rejected the climate finance deal agreed at COP29.\nDeadlock on Mitigation Work Programme (MWP): Due to a divergence in views on the role of fossil fuels in the future energy mix.\nThe Global Stocktake, a comprehensive assessment of global progress towards climate goals, was a point of contention.\nLimited progress on climate targets:  Progress in ambition and action since the initial NDCs plateaued and countries are still off track to deliver on the globally insufficient mitigation pledges for 2030. \nGlobal greenhouse gas emissions set a new record in 2023, with a 1.3% increase from 2022 levels (UNEP's Emissions Gap Report 2024).\nOther Issues:\nSlow pace of operationalizing the Loss and Damage Fund (

In [56]:
quiz=response.get("quiz")

In [62]:

quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [63]:

quiz_table_data


[{'MCQ': 'What was the main reason for developing countries, including India, rejecting the climate finance deal agreed at COP29?',
  'Choices': 'a: Lack of interest in climate action | b: Inadequate funding | c: Disagreement on emission reduction targets | d: Political reasons',
  'Correct': 'b'},
 {'MCQ': 'What was a point of contention during the Global Stocktake, a comprehensive assessment of global progress towards climate goals?',
  'Choices': 'a: Role of fossil fuels in the future energy mix | b: Renewable energy sources | c: Carbon capture technology | d: Climate adaptation strategies',
  'Correct': 'a'},
 {'MCQ': 'What is the reason for the postponement of the next round of Nationally Determined Contributions (NDCs) ahead of COP30?',
  'Choices': 'a: Lack of interest in climate action | b: Disagreement on emission reduction targets | c: Inadequate funding for climate finance | d: Alleged influence of Fossil fuel lobbyists',
  'Correct': 'b'},
 {'MCQ': 'What was the main issue 

In [66]:
quiz=pd.DataFrame(quiz_table_data)

In [67]:
quiz.to_csv("cop29.csv",index=False)